In [1]:
%matplotlib inline
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn import linear_model

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import lightgbm as lgbm

from keras.models import Sequential
from keras.layers import Dense

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import datetime 
import pickle
import os

/usr/local/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.
/usr/local/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a s

In [2]:
vids = pd.read_csv('vids.csv')
vids = vids.dropna()
X = vids[vids.category_id.notna()]
X = X.drop(([
     'video_id', 'channel_title', 'trending_date', 'comments_disabled',
     'comment_count', 'likes', 'dislikes', 'video_error_or_removed', 'thumbnail_link',
     'ratings_disabled']), 1)
Y = vids.views

**Dummy encode categories**

In [3]:
encoded_cateogries = pd.get_dummies(X.category_id, prefix="category_id")
X = X.drop(['category_id'], 1)
for column in encoded_cateogries.columns:
    X[column] = encoded_cateogries[column]

**Publish Date to Publish Day and Encode**

In [4]:
def publish_time_to_day(day_string):
    try:
        split = day_string.split('T')[0].split('-')
        return datetime.datetime(int(split[0]), int(split[1]), int(split[2])).strftime('%A')
    except:
        return 'Monday'

X['publish_day'] = X.publish_time.apply(publish_time_to_day)
encoded_cateogries = pd.get_dummies(X.publish_day, prefix="publish_day")
for column in encoded_cateogries.columns:
    X[column] = encoded_cateogries[column]
X = X.drop(['publish_time', 'publish_day'], 1)

**Encode words with w2v model**

In [5]:
processed_titles = []
processed_descriptions = []
processed_tags = []

for i in tqdm(range(len(X))):
    try:
        processed_titles.append(simple_preprocess(X.title.iloc[i]))
    except:
        processed_titles.append([""])

    try:
        processed_descriptions.append(simple_preprocess(X.description.iloc[i]))
    except:
        processed_descriptions.append([""])

    try:
        processed_tags.append(simple_preprocess(X.tags.iloc[i]))
    except:
        processed_tags.append([""])

In [6]:
w2v_model = Word2Vec.load(os.path.join('mdl', 'word2vec.model'))
all_one_feature = []
dscription_feature = []
title_feature = []
tags_feature = []

for i in tqdm(range(len(X))):
    filtered = [word for word in processed_titles[i] if word in w2v_model.wv.vocab]
    if len(filtered) > 0:
        title_vec = np.mean(w2v_model.wv[filtered], axis=0)
    else:
        title_vec = np.zeros(w2v_model.vector_size)
        
    filtered = [word for word in processed_descriptions[i] if word in w2v_model.wv.vocab]
    if len(filtered) > 0:
        description_vec = np.mean(w2v_model.wv[filtered], axis=0)
    else:
        description_vec = np.zeros(w2v_model.vector_size)
        
    filtered = [word for word in processed_tags[i] if word in w2v_model.wv.vocab]
    if len(filtered) > 0:
        tags_vec = np.mean(w2v_model.wv[filtered], axis=0)
    else:
        tags_vec = np.zeros(w2v_model.vector_size)
    
    all_one_feature.append(np.mean([title_vec, description_vec, tags_vec], axis=0))
    dscription_feature.append(description_vec)
    title_feature.append(title_vec)
    tags_feature.append(tags_vec)

all_one_feature = np.array(all_one_feature)
dscription_feature = np.array(dscription_feature)
title_feature = np.array(title_feature)
tags_feature = np.array(tags_feature)

In [7]:
X = X.drop(['title', 'tags', 'description'], 1)

for i in tqdm(range(w2v_model.vector_size)):
    X[f'word_encodings{i}'] = all_one_feature[:,i]
    X[f'description{i}'] = dscription_feature[:, i]
    X[f'title{i}'] = title_feature[:,i]
    X[f'tags{i}'] = tags_feature[:,i]

**Split data**

In [8]:
X.dropna()
train, test = train_test_split(X, test_size=0.2)

train_y = train.views
train_x = train.drop('views', 1)
test_y = test.views
test_x = test.drop('views', 1)

In [9]:
to_drop = np.array([[f'title{i}', f'tags{i}', f'description{i}'] for i in range(w2v_model.vector_size)]).flatten()
train_x_all = train_x.drop(to_drop,1)
test_x_all = test_x.drop(to_drop,1)

to_drop = [f'word_encodings{i}' for i in range(w2v_model.vector_size)]
train_x_sep = train_x.drop(to_drop,1)
test_x_sep = test_x.drop(to_drop,1)

**Store Train/Test Split**

We may use it for some future analysis.

In [10]:
if not os.path.exists(os.path.join('data')):
    os.mkdir(os.path.join('data'))
    
train_x_all.to_csv(os.path.join('data','train_x_all.csv'), index=False)
train_x_sep.to_csv(os.path.join('data','train_x_sep.csv'), index=False)

test_x_all.to_csv(os.path.join('data','test_x_all.csv'), index=False)
test_x_sep.to_csv(os.path.join('data','test_x_sep.csv'), index=False)

train_y.to_csv(os.path.join('data','train_y.csv'), index=False)
test_y.to_csv(os.path.join('data','test_y.csv'), index=False)

**Train**

In [11]:
def single_layer_network(input_dim):
    model = Sequential()
    model.add(Dense(input_dim, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [12]:
def five_layer_network(input_dim):
    model = Sequential()
    model.add(Dense(input_dim, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
    model.add(Dense(int(input_dim/4), input_dim=input_dim, kernel_initializer='normal', activation='relu'))
    model.add(Dense(int(input_dim/8), input_dim=int(input_dim/4), kernel_initializer='normal', activation='relu'))
    model.add(Dense(int(input_dim/16), input_dim=int(input_dim/8), kernel_initializer='normal', activation='relu'))
    model.add(Dense(int(input_dim/32), input_dim=int(input_dim/16), kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [13]:
lr_all = LinearRegression()
lr_sep = LinearRegression()

svr_all = LinearSVR(C=1.0, epsilon=0.2)
svr_sep = LinearSVR(C=1.0, epsilon=0.2)

rfr_all = RandomForestRegressor(n_estimators=20, max_depth=3)
rfr_sep = RandomForestRegressor(n_estimators=20, max_depth=3)

xgb_all = XGBRegressor(n_estimators=300)
xgb_sep = XGBRegressor(n_estimators=300)

gbm_all = lgbm.LGBMRegressor()
gbm_sep = lgbm.LGBMRegressor()

nn1_all = single_layer_network(train_x_all.shape[1])
nn1_sep = single_layer_network(train_x_sep.shape[1])

nn5_all = five_layer_network(train_x_all.shape[1])
nn5_sep = five_layer_network(train_x_sep.shape[1])

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
print('training lr_all')
lr_all.fit(train_x_all, train_y)

print('training lr_sep')
lr_sep.fit(train_x_sep, train_y)

print('training svr_all')
svr_all.fit(train_x_all, train_y)

print('training svr_sep')
svr_sep.fit(train_x_sep, train_y)

print('training rfr_all')
rfr_all.fit(train_x_all, train_y)

print('training rfr_sep')
rfr_sep.fit(train_x_sep, train_y)

print('training xgb_all')
xgb_all.fit(train_x_all, train_y)

print('training xgb_sep')
xgb_sep.fit(train_x_sep, train_y)

print('training lgbm_all')
gbm_all.fit(train_x_all, train_y)

print('training lgbm_sep')
gbm_sep.fit(train_x_sep, train_y)

print('training nn1_all')
nn1_all.fit(train_x_all, train_y, epochs=50)

print('training nn1_sep')
nn1_sep.fit(train_x_sep, train_y, epochs=50)

print('training nn5_all')
nn5_all.fit(train_x_all, train_y, epochs=50)

print('training nn5_sep')
nn5_sep.fit(train_x_sep, train_y, epochs=50)

training lr_all
training lr_sep
training svr_all
training svr_sep
training rfr_all
training rfr_sep
training xgb_all
training xgb_sep
training lgbm_all
training lgbm_sep
training nn1_all
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
32232/32232 [==============================] - 3s 104us/step - loss: 61611376799732.8203
Epoch 2/50
32232/32232 [==============================] - 3s 91us/step - loss: 61417170265521.8516
Epoch 3/50
32232/32232 [==============================] - 3s 92us/step - loss: 61085405677309.2656
Epoch 4/50
32232/32232 [==============================] - 3s 98us/step - loss: 60665985086563.3750
Epoch 5/50
32232/32232 [==============================] - 3s 93us/step - loss: 60192220496754.4375
Epoch 6/50
32232/32232 [==============================] - 3s 97us/step - loss: 59691728231563.5312
Epoch 7/50
32232/32232 [==============================] - 3s 92us/step - loss: 59183032958943.4688
Epoch 8/50
32232/32232 [==============================] - 3s 92us/step 

32232/32232 [==============================] - 6s 178us/step - loss: 52315603843871.5781
Epoch 29/50
32232/32232 [==============================] - 6s 186us/step - loss: 52150365442982.2812
Epoch 30/50
32232/32232 [==============================] - 6s 185us/step - loss: 51991376714980.9922
Epoch 31/50
32232/32232 [==============================] - 6s 179us/step - loss: 51836118092318.3750
Epoch 32/50
32232/32232 [==============================] - 6s 184us/step - loss: 51686358505204.1172
Epoch 33/50
32232/32232 [==============================] - 6s 185us/step - loss: 51544208505562.4531
Epoch 34/50
32232/32232 [==============================] - 6s 185us/step - loss: 51407093922010.0703
Epoch 35/50
32232/32232 [==============================] - 6s 178us/step - loss: 51273913046271.9375
Epoch 36/50
32232/32232 [==============================] - 6s 184us/step - loss: 51145862128736.3281
Epoch 37/50
32232/32232 [==============================] - 6s 185us/step - loss: 51021507127014.8984
Ep

32232/32232 [==============================] - 8s 247us/step - loss: 22034580188493.7109
Epoch 10/50
32232/32232 [==============================] - 8s 258us/step - loss: 20954878159481.8711
Epoch 11/50
32232/32232 [==============================] - 8s 253us/step - loss: 19789603087496.9922
Epoch 12/50
32232/32232 [==============================] - 8s 251us/step - loss: 18981632161721.3477
Epoch 13/50
32232/32232 [==============================] - 8s 252us/step - loss: 17999278399701.4922
Epoch 14/50
32232/32232 [==============================] - 8s 251us/step - loss: 17403147672528.4727
Epoch 15/50
32232/32232 [==============================] - 8s 253us/step - loss: 16825822633058.8672
Epoch 16/50
32232/32232 [==============================] - 8s 258us/step - loss: 16153962527373.1855
Epoch 17/50
32232/32232 [==============================] - 8s 247us/step - loss: 15756827235364.3457
Epoch 18/50
32232/32232 [==============================] - 8s 255us/step - loss: 15199594532220.4746
Ep

**save models**

In [15]:
pickle.dump(lr_all, open(os.path.join('mdl', 'lr_all.pkl'), 'wb'))
pickle.dump(lr_sep, open(os.path.join('mdl', 'lr_sep.pkl'), 'wb'))
pickle.dump(svr_all, open(os.path.join('mdl', 'svr_all.pkl'), 'wb'))
pickle.dump(svr_sep, open(os.path.join('mdl', 'svr_sep.pkl'), 'wb'))
pickle.dump(rfr_all, open(os.path.join('mdl', 'rfr_all.pkl'), 'wb'))
pickle.dump(rfr_sep, open(os.path.join('mdl', 'rfr_sep.pkl'), 'wb'))
pickle.dump(xgb_all, open(os.path.join('mdl', 'xgb_all.pkl'), 'wb'))
pickle.dump(xgb_sep, open(os.path.join('mdl', 'xgb_sep.pkl'), 'wb'))
pickle.dump(gbm_all, open(os.path.join('mdl', 'gbm_all.pkl'), 'wb'))
pickle.dump(gbm_sep, open(os.path.join('mdl', 'gbm_sep.pkl'), 'wb'))
pickle.dump(nn1_all, open(os.path.join('mdl', 'nn1_all.pkl'), 'wb'))
pickle.dump(nn1_sep, open(os.path.join('mdl', 'nn1_sep.pkl'), 'wb'))
pickle.dump(nn5_all, open(os.path.join('mdl', 'nn5_all.pkl'), 'wb'))
pickle.dump(nn5_sep, open(os.path.join('mdl', 'nn5_sep.pkl'), 'wb'))